## 首先判断是否是相关内容
    方法1：模型判断
        Yi-34B的效果很好（个人测试至少ChatGPT同水准，部分甚至超越GPT-4水准），可用于用户的意图识别。
    *方法2: 构建FAQ向量库做判断
        基于相似度对比来进行分类

地址：[01-ai/Yi-34B · Hugging Face](https://huggingface.co/01-ai/Yi-34B)


# 直接让模型做决定
## Yi-34B测试案例

query1：
任务一：将{}中句子的实体和属性提取出来：{百日咳的易感人群和治好概率}
任务二：判别{}中句子的意图为列表中的哪一项，[医疗，数学，物理，其他]

answer1：
任务一:将中句子的实体和属性提取出来:{百日咳的易感人群和治好概率}
答案:实体:百日咳,属性:易感人群,治好概率
任务二:判别中句子的意图为列表中的哪一项，[医疗，数学，物理，其他]
答案: 医疗

query2:
任务一：将{}中句子的实体和属性提取出来：{哪些人易感染肺放线菌病}
任务二:判别中句子的意图为列表中的哪一项，[医疗，数学，物理，其他]

answer2:
任务一:将中句子的实体和属性提取出来:[哪些人易感染肺放线菌病)答案:实体:肺放线菌病,属性:易感人群
任务二:判别中句子的意图为列表中的哪一项，[医疗，数学，物理，其他]答案:医疗

query3:
任务一:将中句子的实体和属性提取出来:[治愈率大于30%,感染概率小于50%的疾病有哪些,至少列出3个)
任务二:判别中句子的意图为列表中的哪一项，[医疗，数学，物理，其他]

answer3:
任务一：将{}中句子的实体和属性提取出来:[治愈率大于30%,感染概率小于50%的疾病有哪些,至少列出3个)
答案：由于这个句子没有明确的疾病名称，所以无法提取实体。属性有治愈率和感染概率。
任务二：判别{}中句子的意图为列表中的哪一项，[医疗，数学，物理，其他]
答案: 医疗


In [ ]:
# 向量召回来判断
# 通过prompt来选择 下面的模板可以换为更相似的FAQ文件，或者更容易比较的问题、文件模板

from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import PromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableLambda, RunnablePassthrough
from langchain.utils.math import cosine_similarity
from api_key import OPENAI_API_KEY

physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise and easy to understand manner. \
When you don't know the answer to a question you admit that you don't know.

Here is a question:
{query}"""

math_template = """You are a very good mathematician. You are great at answering math questions. \
You are so good because you are able to break down hard problems into their component parts, \
answer the component parts, and then put them together to answer the broader question.

Here is a question:
{query}"""

medical_template = """You are a highly skilled physician. You excel at diagnosing medical conditions. \
You are adept because you can dissect complex medical cases into their underlying symptoms, \
analyze these symptoms, and then synthesize them to arrive at a comprehensive diagnosis.

Here is a case for review:
{query}"""

universal_template = """You are an expert in your field. You excel at addressing complex questions and problems. \
Your expertise is evident in your ability to deconstruct intricate issues into their fundamental elements, \
solve these individual elements, and then integrate them to form a holistic solution or answer.

Here is a question or problem for your consideration:
{query}"""

embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)
prompt_templates = [physics_template, math_template, medical_template, universal_template]
prompt_embeddings = embeddings.embed_documents(prompt_templates)



def prompt_router(input):
    query_embedding = embeddings.embed_query(input["query"])
    similarity = cosine_similarity([query_embedding], prompt_embeddings)[0]
    most_similar = prompt_templates[similarity.argmax()]
    # print("Using MATH" if most_similar == math_template else "Using PHYSICS")
    if most_similar == math_template:
        print("Using MATH")
    elif most_similar == physics_template:
        print("Using PHYSICS")
    elif most_similar == medical_template:
        print("Using MEDICAL")
    elif most_similar == universal_template:
        print("Using UNIVERSAL")

prompt_router({"query": "百日咳的易感人群?"})        

### 如果相关，如何识别意图

In [ ]:
# 意图识别

# 方法一：向量召回
# FAQ文件匹配意图识别
# 参考amazen的知识问答效果https://github.com/aws-samples/private-llm-qa-bot/blob/5140a163ac833fb71ee765bd2f2ad779dc3a6fe9/docs/intentions/aws_faq.example#L24

# 方法二：模型识别
# 目前的Yi-34B模型的效果已经很好，可以考虑利用上述行业内FAQ文件直接微调模型，使用微调后的模型直接进行识别意图



#### 意图缺失补全

In [ ]:
# 意图补全
# 方法一：向量召回和模型结合
# 先通过向量相似匹配到相似的问题，然后再将其合并输入给模型，模型输出需要补全的内容。

# query1:
# 我的问题为{query：的治愈率},请根据下方{}内的相似示例提示我我的问题缺失哪些部分？{query:百日咳的治愈概率？}
# answer1:
# 你的问题缺失了疾病名称。

# query2:
# 我的问题为{query："治愈率大于30%,感染概率小于的疾病有哪些,至少列出3个"},请根据接下来{}内的相似示例提示我我的问题缺失哪些部分？{query:请告诉我感染率在30%到50%区间的疾病}
# answer2:
#你的问题缺失了一个明确的治愈率数值和一个具体的感染概率范围。
